In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import norm

## implied volatility for European put option

In [8]:
def EuropeanPut(sigma,S=10,T=1,r=0.02,K=9):
    d1 = (math.log(S/K)+(r+0.5*sigma**2)*T)/(sigma*math.sqrt(T))
    d2 = d1-sigma*math.sqrt(T)
    p = K*math.exp(-r*T)*norm.cdf(-d2)-S*norm.cdf(-d1)
    return p

In [9]:
def vega(sigma,S=10,T=1,r=0.02,K=9):
    d1 = (math.log(S/K)+(r+0.5*sigma**2)*T)/(sigma*math.sqrt(T))
    vega = S*math.sqrt(T)*norm.pdf(d1)
    return vega

In [10]:
def NewtonMethod(f,Df,target,x0,max_iter,tol):
    xn = x0
    for n in range(0,max_iter):
        fxn = f(xn)-target
        if abs(fxn) < tol:
            print('Found solution after',n,'iterations.')
            return xn
        Dfxn = Df(xn)
        if Dfxn == 0:
            print('Zero derivative. No solution found.')
            return None
        xn = xn - fxn/Dfxn
    print('Exceeded maximum iterations. No solution found.')
    return None

In [11]:
NewtonMethod(EuropeanPut,vega,2,0.5,100,0.001)

Found solution after 2 iterations.


0.6925759283015839

## implied volatility for American put option with Crank-Nicolson

In [2]:
def TDMA(a,b,c,d):
    n = len(d)
    x = [0 for _ in range(n)]
    
    c[0] = c[0]/b[0]
    d[0] = d[0]/b[0]
    
    for i in range(1,n-1):
        temp = b[i] -a[i]*c[i-1]
        c[i] = c[i]/temp
        d[i] = (d[i]-a[i]*d[i-1])/temp
        
    d[n-1]=(d[n-1]-a[n-1]*d[n-2])/(b[n-1]-a[n-1]*c[n-2])
    
    x[-1] = d[-1]
    for i in range(n-2,-1,-1):
        x[i] = d[i]-c[i]*x[i+1]
    
    return x 

In [17]:
# 输入sigma 输出一条期权价格关于股票价格的曲线
def CN_AmericanPut(T=1,r=0.02,sigma=0.68,K=9,N=100,M=200,U=100,L=0):
    X_max = np.log(U) + (r-0.5*sigma**2)*T
    X_min = np.log(L+0.01) + (r-0.5*sigma**2)*T
    t0 = 0.5*sigma**2*T
    delta_t = t0/N
    delta_y = (X_max-X_min)/M
    lamda = delta_t/(delta_y**2)
    
    S = [np.exp(i*delta_y+X_min-(r-0.5*sigma**2)*T) for i in range(0,M+1,1)]
    time = [delta_t*i for i in range(0,N+1,1)]
    X = np.array([i*delta_y+X_min for i in range(0,M+1,1)])
    u = np.array([math.exp(-r*T)*max(0,K-math.exp(i*delta_y+X_min)) for i in range(0,M+1,1)])
    print(S)
    
    A = [-lamda for _ in range(1,M,1)]
    B = [2+2*lamda for _ in range(1,M,1)]
    C = [-lamda for _ in range(1,M,1)]

    A[0] = 0
    C[-1] = 0

    for n in range(N):
        
        D = [lamda*u[i-1]+(2-2*lamda)*u[i]+lamda*u[i+1] for i in range(1,len(u)-1)]
        D[0] += lamda*K*np.exp(-r*(T-2*n*delta_t/(sigma**2)))
            
        u[1:-1] = TDMA(A[:],B[:],C[:],list(D))
        u[0] = K*np.exp(-r*(T-2*n*delta_t/(sigma**2)))
        u[-1] = 0
        
        u0 = np.exp(-r*(T-2*n*delta_t/(sigma**2)))*np.maximum(0,K-np.exp(X-(2*r/(sigma**2)-1)*n*delta_t))
        u = np.maximum(u,u0)
        

    u = u*np.exp(r*(T-2*t0/(sigma**2)))
    

    return u,S
    

In [13]:
# 输入sigma和股票价格 输出期权价格
def encap_CN_AmericanPut(sigma,stock_price):
#     crank-nicolson scheme
    u,S = CN_AmericanPut(T=1,r=0.02,sigma=sigma,K=9,N=100,M=200,U=100,L=0)
    
#     implicit scheme
#     [u,S,_,_] = AmericanPut(T=1,r=0.02,sigma=sigma,K=9,N=100,M=200,R=100)
    
    i = (np.abs(np.array(S)-stock_price)).argmin()
    return u[i]

In [14]:
def secant(func,target,x1,x2,stock_price,tol):
    # x1:close to 0     
    # x2:European option implied volatility
    
    f1 = func(x1,stock_price)-target
    f2 = func(x2,stock_price)-target
    
    if f1*f2 > 0:
        raise RuntimeError('reset x1,x2 please')

    while True:
        f1 = func(x1,stock_price)-target
        f2 = func(x2,stock_price)-target
        
        x = x1-f1*(x1-x2)/(f1-f2)
        fx = func(x,stock_price)-target
        print(x)
        
        if f1*fx<0:
            x1 = x1
            x2 = x
        elif f2*fx<0:
            x1 = x
            x2 = x2
        else:
            x2 = x
            x1 = x
        if abs(fx)<=tol:
            break
    return x

In [15]:
upperBoundary = NewtonMethod(EuropeanPut,vega,2,0.5,100,0.001)
stock_price = 10
option_price = 2
tol = 0.1
x1 = 0.000001
x2 = upperBoundary
secant(encap_CN_AmericanPut,option_price,x1,x2,stock_price,tol)

Found solution after 2 iterations.
0.6863302823662174


0.6863302823662174

## implicit scheme

In [ ]:
def AmericanPut(T=1,r=0.02,sigma=0.666,K=9,N=100,M=200,R=100):
    delta_t = T/N
    delta_y = R/M
    time = [delta_t*i for i in range(0,N+1,1)]
    u = np.array([max(0,K-i*delta_y) for i in range(0,M+1,1)])
    u0 = u.copy()
    S0 = np.array([i*delta_y for i in range(0,M+1,1)])
    boundary = [K]

    A = [-0.5*delta_t*(sigma*i)**2 for i in range(1,M,1)]
    B = [1+delta_t*(r+r*i+(i*sigma)**2) for i in range(1,M,1)]
    C = [delta_t*(-r*i-0.5*(i*sigma)**2) for i in range(1,M,1)]
    
    A[0] = 0
    C[-1] = 0

    
    for n in range(N):
        D = u[1:-1].copy()
        D[0] -= -0.5*delta_t*sigma**2*K
        u[1:-1] = TDMA(A[:],B[:],C[:],list(D))
        u[0] = np.exp(-r*(N-n)*delta_t)*K
        u[-1] = 0
        u = np.maximum(u,u0)
        boundary.append(S0[((u-u0)!=0).argmax(axis=0)-1])
        
        
    return [u,S0,boundary,time]

In [ ]:
[u,S0,boundary,time] = AmericanPut()
stock_price = 10
u[(np.abs(np.array(S0)-stock_price)).argmin()]

In [ ]:
plt.xlabel("price of underlying asset")
plt.ylabel("price of option")
plt.title("American Put Price vs. Underlying Asset Price")
_ = plt.plot(S0,u)